In [36]:
import tensorflow as tf
import numpy as np
import cv2

import warnings
warnings.filterwarnings('ignore')  # 不打印 warning 

## VGG16网络结构

In [37]:
data_dict = np.load('./vgg16.npy', encoding='latin1').item()

VGG_MEAN = [103.939, 116.779, 123.68]

FileNotFoundError: [Errno 2] No such file or directory: './vgg16.npy'

In [38]:
def get_conv_filter(name):
    return tf.constant(data_dict[name][0], name='filter')

In [39]:
def get_biases(name):
    return tf.constant(data_dict[name][1], name='biases')

In [40]:
def get_fc_weight(name):
    return tf.constant(data_dict[name][0], name='weights')

In [41]:
def conv_layer(x_input, name):
    with tf.variable_scope(name):
        filt = get_conv_filter(name)
        
        layer = tf.nn.conv2d(x_input, filt, [1, 1, 1, 1], padding='SAME')
        
        conv_biases = get_biases(name)
        layer = tf.nn.bias_add(layer, conv_biases)
        
        layer = tf.nn.relu(layer)
        return layer

In [42]:
def fc_layer(x_input, name):
    with tf.variable_scope(name):
        shape = x_input.get_shape().as_list()
        dim = 1
        for d in shape[1:]:
            dim *= d
        x = tf.reshape(x_input, [-1, dim])
        
        w = get_fc_weight(name)
        b = get_biases(name)
        
        fc = tf.nn.bias_add(tf.matmul(x, w), b)
    return fc

In [43]:
def max_pool_layer(x_input, name):
    return tf.nn.max_pool(x_input, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name=name)

In [44]:
def avg_pool_layer(x_input, name):
    return tf.nn.avg_pool(x_input, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name=name)

In [45]:
def vgg16(rgb):
    rgb_scaled = rgb * 255.0
    
    # Convert rgb to bgr
    r, g, b = tf.split(axis=3, num_or_size_splits=3, value=rgb_scaled)
    assert r.get_shape().as_list()[1:] == [224, 224, 1]
    assert g.get_shape().as_list()[1:] == [224, 224, 1]
    assert b.get_shape().as_list()[1:] == [224, 224, 1]
    bgr = tf.concat(axis=3, values=[
        b - VGG_MEAN[0],
        g - VGG_MEAN[1],
        r - VGG_MEAN[2],
    ])
    assert bgr.get_get_shape().as_list()[1:] == [224, 224, 3]
    
    conv1_1 = conv_layer(bgr, 'conv1_1')
    conv1_2 = conv_layer(conv1_1, 'conv1_2')
    pool1 = max_pool_layer(conv1_2, 'pool1')
    
    # 
    conv2_1 = conv_layer(pool1, 'conv2_1')
    conv2_2 = conv_layer(conv2_1, 'conv2_2')
    pool2 = max_pool_layer(conv2_2, 'pool2')
    
    #
    conv3_1 = conv_layer(pool2, 'conv3_1')
    conv3_2 = conv_layer(conv3_1, 'conv3_2')
    conv3_3 = conv_layer(conv3_2, 'conv3_3')
    pool3 = max_pool_layer(conv3_3, 'pool3')
    
    #
    conv4_1 = conv_layer(pool3, 'conv4_1')
    conv4_2 = conv_layer(conv4_1, 'conv4_2')
    conv4_3 = conv_layer(conv4_2, 'conv4_3')
    pool4 = max_pool_layer(conv4_3, 'pool4')
    
    #
    conv5_1 = conv_layer(pool4, 'conv5_1')
    conv5_2 = conv_layer(conv5_1, 'conv5_2')
    conv5_3 = conv_layer(conv5_2, 'conv5_3')
    pool5 = max_pool_layer(conv5_3, 'pool5')
    
    #
    fc6 = fc_layer(pool, 'fc6')
    assert fc6.get_shape().as_list()[1:] == [4096]
    relu6 = tf.nn.relu(fc6)
    
    fc7 = fc_layer(relu7, 'fc7')
    relu7 = tf.nn.relu(fc7)
    
    fc8 = fc_layer(relu7, 'fc8')
    
    prob = tf.nn.softmax(fc8, name='prob')
    
    return prob

## 训练与测试

### 数据

In [46]:
img = cv2.imread('./data/tiger.jpeg')
print('原始尺寸：', img.shape)
img = cv2.resize(img, (224, 224), interpolation=cv2.INTER_CUBIC)
print('变换后尺寸：', img.shape)
batch = img.reshape((1, 224, 224, 3))

one_hot_label = [[1 if i == 292 else 0 for i in range(1000)]]

原始尺寸： (166, 304, 3)
变换后尺寸： (224, 224, 3)


In [47]:
n_step = 10
n_show_step = 1

with tf.Session() as sess:
    images = tf.placeholder(tf.float32, [None, 224, 224, 3])
    ground_truth = tf.placeholder(tf.float32, [None, 1000])
    
    prob = vgg16(images)
    loss = tf.reduce_sum((prob - ground_truth) ** 2)
    train = tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(loss)
    
    sess.run(tf.global_variables_initializer())
    for i in range(n_step):
        sess.run(train, feed_dict={images: batch, ground_truth: one_hot_label})
        if i % n_show_step == 0:
            print(sess.run(loss, feed_dict={images: batch, ground_truth: one_hot_label}))
    

AttributeError: 'Tensor' object has no attribute 'get_get_shape'